## Import packages:

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import nltk 
import gensim
import re
import unicodedata
import datetime
import collections
import random
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import treetaggerwrapper
import string
import os
import platform
treetaggerPath = str(os.getcwd()) + "/treetagger/" + str(platform.system()) + "/"

In [2]:
treetaggerPath

'/Users/albertomariopirovano/Documents/Programming/python/notebooks/tis_notebooks/treetagger/Darwin/'

## Load dataset:

In [3]:
trains24 = pd.read_csv("./Dataset/articoliS24O.csv", delimiter="\t")
trainRadiocor = pd.read_csv("./Dataset/articoliRadiocor.csv", delimiter='\t')

colsel_trains24 = trains24[['identificativo', 'body']]

colsel_trainRadiocor = trainRadiocor[['identificativo', 'body']]

colsel_merged = pd.concat([colsel_trains24, colsel_trainRadiocor])

colsel_merged.dropna(inplace=True)
colsel_merged = colsel_merged.reset_index()
colsel_merged = colsel_merged.drop('index', axis=1)

## Clean dataset:

In [4]:
print("SHAPE BEFORE REMOVING DUPLICATES: " + str(colsel_merged.shape)) 
colsel_merged.drop_duplicates(inplace=True) 
print("SHAPE AFTER REMOVING DUPLICATES: " + str(colsel_merged.shape))

colsel_merged['body'] = colsel_merged['body'].apply(lambda x : str(x).lower())
colsel_merged['body'] = colsel_merged['body'].str.replace(r'(\\)+[a-z]\{[0-9]+\}','')
colsel_merged['body'] = colsel_merged['body'].str.replace('\n','')

#how to deal with times?
#colsel_merged['body'] = colsel_merged['body'].str.replace('[0-9]+\.[0-9]+','')

#handle percentages
colsel_merged['body'] = colsel_merged['body'].apply(lambda x: str(re.sub(r'([0-9]+)((\,|\.)[0-9]+)?((\s)?per(\s)?cento|(\s)?%)',r'\1_percento',str(x))))

colsel_merged['body'] = colsel_merged['body'].str.replace(r'il( )?sole( )?24( )?ore','sole24ore')
colsel_merged['body'] = colsel_merged['body'].str.replace('(milion(i|e)|miliard(i|o)|euro)','')
#remove useless endings
colsel_merged['body'] = colsel_merged['body'].str.replace(r'\(c.fe.\)','')
colsel_merged['body'] = colsel_merged['body'].str.replace(r'((\()?( )?sole24ore (radiocor)?( plus)?( )?(\))?.\-.*?)\-','')
colsel_merged['body'] = colsel_merged['body'].str.replace(r'(continua (da )?pagina [0-9]+)© riproduzione riservata(continua (da )?pagina [0-9]+)', '')
colsel_merged['body'] = colsel_merged['body'].str.replace(r'((© )?riproduzione riser(vata)?).*','')

#format urls
colsel_merged['body'] = colsel_merged['body'].str.replace(r'(www\.|http\:\/\/|https\:\/\/|\.com|\.net|\.org|\.it|@[a-zA-Z]+)','')
colsel_merged['body'] = colsel_merged['body'].str.replace(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?','')  

#format dots for pos_tagging, done in order to mess up with urls!
colsel_merged['body'] = colsel_merged['body'].str.replace(r':',' : ')
colsel_merged['body'] = colsel_merged['body'].str.replace(r'.','. ')
colsel_merged['body'] = colsel_merged['body'].str.replace(r'( )+',' ')

#adjusted html tags cleaning! 2 is the magic number
for _ in np.arange(2):
    colsel_merged['body'] = colsel_merged['body'].apply(lambda x: str(BeautifulSoup(x,"lxml").text))

#removing unprintable unicode
colsel_merged['body'].replace('\u200e','',regex=True,inplace=True)

SHAPE BEFORE REMOVING DUPLICATES: (10150, 2)
SHAPE AFTER REMOVING DUPLICATES: (9283, 2)


In [5]:
tmp= colsel_merged[colsel_merged['identificativo']=='FM20151009036ACmwyfCB']['body'].values
print(tmp)

[ "pietro scott jovane lascia la poltrona di amministratore delegato di rcsmedia group. al termine di una giornata fitta di incontri e partita in prima mattinata con il summit con le banche creditrici per la rinegoziazione del debito, il consiglio di amministrazione del gruppo editoriale ha comunicato l'uscita «consensuale» di scott jovane a partire dal prossimo 15 ottobre e l'assegnazione delle deleghe di jovane ad interim al presidente maurizio costa. «il cda di rcs mediagroup - si legge in una nota - ha preso atto della volontà dell'ad pietro scott jovane di lasciare il proprio incarico, ritenendo concluso un ciclo della vita aziendale e manifestando l'intenzione di intraprendere un nuovo percorso professionale».  inoltre il consiglio ha comunicato che per la risoluzione del contratto saranno corrisposte 150 mila al manager, a cui si sommano 600 mila lordi per il patto di non concorrenza. il board, iniziato alle 17. 30 in via rizzoli, nella sede centrale di rcs mediagroup, dopo la r

In [6]:
print(colsel_merged[colsel_merged['identificativo']=='FM20160827022ADDGKkAB']['body'].values)

[ 'la saga continua. nonostante che tutti sappiano come va a finire. ma il commissario peo che si occupa di policy digitale, günther oettinger, non ne è al corrente, evidentemente. la saga è quella degli editori che si sentono defraudati di un diritto a causa dei motori di ricerca - cioè di google - che propongono ai lettori qualche riga dei loro articoli insieme ai link per trovarli online. oettinger sta studiando una proposta di direttiva che dà agli editori il diritto di chiedere un compenso per quelle righe di testo. forse pensa così di apparire come l’eroe della saga, l’innovatore che risolve un problema a lungo dibattuto. invece, arriva ultimo. e dopo che questo genere di soluzione è già fallito in spagna e in germania. ma è veramente possibile che non lo sappia? e il vice presidente della commissione, andrus ansip, che conosce il digitale molto meglio di oettinger e che si occupa del mercato unico digitale, non gli ha detto nulla? in realtà, la realtà digitale va troppo veloce p

In [7]:
def clean_tag(tt):
    if(len(tt)<3):
        print(tt[0])
    if(tt[2]=='@card@' or tt[2]=='@ord@'):
        return tt[0]
    else:
        return tt[2]

In [8]:
def pos_tagger_ita(lst):
    corpus = {}
    stopw = set(stopwords.words("italian"))
    puntk = set(string.punctuation)
    rem = stopw | puntk | set('«') | set('»')
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='it', TAGDIR=treetaggerPath)
    for txt in lst:
        tags=tagger.tag_text(txt[1])
        pp_tags= treetaggerwrapper.make_tags(tags)
        tagged_txt = [clean_tag(t) for t in pp_tags if clean_tag(t) not in rem]
        corpus[txt[0]]= tagged_txt
    return corpus

corp = pos_tagger_ita(colsel_merged[['identificativo','body']].values.tolist())

In [9]:
corp['FM20151108016ACcDasOB']

['mondo',
 'biotecnologia',
 'investire',
 'bb',
 'biotech',
 'essere',
 'complesso',
 'comparto',
 'parte',
 'richiedere',
 'specializzazione',
 'dall’altra',
 'seppure',
 'diverso',
 'società',
 'essere',
 'ormai',
 'consolidare',
 'mantenere',
 'caratteristica',
 'frontiera',
 'scientifico',
 'punto',
 'vista',
 'borsistico',
 'andare',
 'maneggiare',
 'cura',
 'certo',
 'lungo',
 'periodo',
 'potenzialità',
 'essere',
 'concretizzare',
 'nasdaq',
 'biotechnology',
 'index',
 'nbi',
 'cioè',
 'paniere',
 'essere',
 'quotare',
 'grande',
 'parte',
 'società',
 'settore',
 'ultimo',
 '10',
 'avere',
 'mettere',
 'segno',
 'incremento',
 'senza',
 'contare',
 're-investimento',
 'cedola',
 '335_percento',
 'chiusura',
 '3',
 '11',
 '2014',
 'l’s',
 'p500',
 'invece',
 'stesso',
 'arco',
 'tempo',
 'contabilizzare',
 'dividendo',
 'essere',
 'salire',
 '105_percento',
 'insomma',
 'migliore',
 'andamento',
 'nbi',
 'rispetto',
 'wall',
 'street',
 'tradizionale',
 'essere',
 'numero',
 

In [13]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)
# Initialize and train the model (this will take some time)
from gensim.models import word2vec

In [82]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 20          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [83]:
print("Training model...")
model = word2vec.Word2Vec(list(corp.values()), workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2017-03-27 19:29:07,713 : INFO : collecting all words and their counts
2017-03-27 19:29:07,729 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2017-03-27 19:29:08,037 : INFO : collected 37998 word types from a corpus of 1300313 raw words and 9283 sentences
2017-03-27 19:29:08,038 : INFO : Loading a fresh vocabulary
2017-03-27 19:29:08,085 : INFO : min_count=40 retains 3849 unique words (10% of original 37998, drops 34149)
2017-03-27 19:29:08,088 : INFO : min_count=40 leaves 1134238 word corpus (87% of original 1300313, drops 166075)
2017-03-27 19:29:08,254 : INFO : deleting the raw counts dictionary of 37998 items
2017-03-27 19:29:08,256 : INFO : sample=0.001 downsamples 35 most-common words
2017-03-27 19:29:08,257 : INFO : downsampling leaves estimated 1054399 word corpus (93.0% of prior 1134238)
2017-03-27 19:29:08,259 : INFO : estimated required memory for 3849 words and 300 dimensions: 11162100 bytes
2017-03-27 19:29:08,282 : INFO : resetting layer weights
2017-03-27 19:29:08,424 : INFO : training model with 4 workers on 3849 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=20
2017-03-27 19:29:0

In [75]:
model.most_similar("quotazione")

[('borsa', 0.8388105630874634),
 ('mta', 0.77818363904953),
 ('star', 0.7731841802597046),
 ('sbarco', 0.7719237804412842),
 ('quotare', 0.7455160617828369),
 ('listino', 0.7436736822128296),
 ('aim', 0.7377946972846985),
 ('titolo', 0.7290970683097839),
 ('matricola', 0.7286973595619202),
 ('telematico', 0.7238479852676392)]

In [85]:
model.most_similar("londra")

[('approdare', 0.7709338665008545),
 ('parigi', 0.7646940350532532),
 ('road', 0.7592347860336304),
 ('day', 0.7529236078262329),
 ('sbarcare', 0.7421725988388062),
 ('debutto', 0.7237395644187927),
 ('show', 0.722076416015625),
 ('francoforte', 0.7215309143066406),
 ('l’annuncio', 0.7210966348648071),
 ('investor', 0.716953456401825)]

In [76]:
model.syn0.shape

2017-03-27 19:00:58,858 : WARNING : direct access to syn0 will not be supported in future gensim releases, please use model.wv.syn0


(3849, 300)

In [81]:
model.most_similar("biotecnologia")

KeyError: "word 'biotecnologia' not in vocabulary"